<a href="https://colab.research.google.com/github/stevenAnto/llm/blob/main/LangChaing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Ingresa tu clave de OpenAI: ")


Ingresa tu clave de OpenAI: ··········


In [ ]:
!pip install langchain langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
llm.invoke("la profesion de IA tiene un futuro prometedor?")

AIMessage(content='Sí, la profesión de Inteligencia Artificial (IA) tiene un futuro muy prometedor. La IA está cada vez más presente en numerosos sectores y aplicaciones, desde la atención médica hasta la industria automotriz. Se espera que el campo de la IA continúe creciendo y evolucionando en los próximos años, lo que generará una demanda creciente de profesionales especializados en esta área. Además, la IA está en constante desarrollo y hay un gran potencial para nuevas aplicaciones y avances en este campo. Por lo tanto, la profesión de IA ofrece grandes oportunidades de crecimiento y desarrollo profesional.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 19, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-012

In [ ]:
from  langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Eres un asistente de IA, puedes responder a preguntas sobre IA"),
    ("user","{input}")
])

In [ ]:
chain  = prompt | llm
chain.invoke({"input":"la profesion de IA tiene un futuro prometedor?"})

AIMessage(content='Sí, la profesión de Inteligencia Artificial tiene un futuro prometedor. Con los rápidos avances en tecnología y el creciente interés en la aplicación de IA en diferentes campos, la demanda de profesionales en esta área está en alza. Se espera que la IA continúe desempeñando un papel importante en la transformación digital de diversas industrias, lo que generará oportunidades laborales y de investigación en el campo de la IA. Es un campo en constante evolución que ofrece un amplio abanico de posibilidades para aquellos interesados en desarrollar una carrera en este sector.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 39, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprin

#Memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI


In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Eres un asistente de {ability}, puedes responder a preguntas sobre {ability}",
     ),
    MessagesPlaceholder(variable_name="history"),#ingresa el historico de la conversacion
    ("human","{input}")
])

In [ ]:
runnable = prompt | model

In [ ]:
!pip install langchain-community


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:
store = {}

In [ ]:
#Para saber si hay contexto anterior
def get_session_history(session_id: str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_message_key="input",
    history_messages_key="history",
)

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"que es una funcion de perdida"},
    config={"configurable":{"session_id":"abc123"}},
    )

AIMessage(content='Una función de pérdida, también conocida como función de costo, es una medida que se utiliza en aprendizaje automático para evaluar qué tan bien se está desempeñando un modelo predictivo en un conjunto de datos de entrenamiento. Esta función calcula la diferencia entre las predicciones del modelo y los valores reales de los datos de entrenamiento. El objetivo es minimizar esta función de pérdida para que el modelo pueda hacer predicciones precisas. Hay diferentes funciones de pérdida que se utilizan en función del tipo de problema de aprendizaje automático que se esté abordando, como regresión, clasificación, etc. Ejemplos de funciones de pérdida comunes incluyen el error cuadrático medio (MSE) para problemas de regresión y la entropía cruzada para problemas de clasificación.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 184, 'prompt_tokens': 40, 'total_tokens': 224, 'completion_tokens_details': {'accepted_prediction_t

In [ ]:
with_message_history.invoke(
    {"ability":"ciencia de datos","input":"como?, no te entendi"},
    config={"configurable":{"session_id":"abc123"}},
    )

AIMessage(content='Una función de pérdida es una herramienta fundamental en el aprendizaje automático que nos ayuda a medir cuán bueno es un modelo en sus predicciones. En pocas palabras, esta función calcula la diferencia entre la predicción que hace nuestro modelo y el valor real que debería predecir en un conjunto de datos de entrenamiento. El objetivo es minimizar esta diferencia, es decir, reducir al máximo la función de pérdida, para que nuestro modelo sea capaz de hacer predicciones más precisas y acertadas. La elección de la función de pérdida depende del tipo de problema que estemos abordando, ya sea regresión, clasificación u otro. Algunas funciones de pérdida comunes son el error cuadrático medio (MSE) para problemas de regresión y la entropía cruzada para problemas de clasificación.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 238, 'total_tokens': 429, 'completion_tokens_details': {'accepted_prediction_

#Tools


In [ ]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=31382311e00fdc27bffca165a61befc1a6355cecbc008cbee55d5b637134657d
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tool.name

'wikipedia'

In [ ]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [ ]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [ ]:
tool.run({"query":"Alan Turing"})

'Page: Alan Turing\nSummary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer sc'

In [ ]:
tool.run({"query":"quien es Alan Turing"})

'Page: Arturo Puig\nSummary: Arturo José Alberto Puig (born 17 November 1944) is an Argentine actor. He has worked in the 1990s success Grande, pá!!!'

#Agents


In [ ]:
!pip install -U langchain-tavily


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 54.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
api_key = getpass(" Ingresa tu Tavily API Key: ")
search = TavilySearchResults(tavily_api_key=api_key)

 Ingresa tu Tavily API Key: ··········


/tmp/ipython-input-29-3490143203.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(tavily_api_key=api_key)


In [ ]:
search.invoke("Que temperatura hay en Londres")

[{'title': 'El clima y el tiempo promedio en todo el año en Londres Reino Unido',
  'url': 'https://es.weatherspark.com/y/45062/Clima-promedio-en-Londres-Reino-Unido-durante-todo-el-a%C3%B1o',
  'content': 'El clima y el tiempo promedio en todo el año en Londres Reino Unido =================================================================== En Londres, los veranos son cortos, cómodos y parcialmente nublados y los inviernos son largos, muy frío, ventosos y mayormente nublados. Durante el transcurso del año, la temperatura generalmente varía de _4 °C_ a _23 °C_ y rara vez baja a menos de _-1 °C_ o sube a más de _29 °C_. En base a la [puntuación de [...] temperatura máxima promedio diaria es más de _20 °C_. El mes más cálido del año en Londres es _julio_, con una temperatura máxima promedio de _23 °C_ y mínima de _15 °C_. La _temporada fresca_ dura _4,0 meses_, del _16 de noviembre_ al _18 de marzo_, y la temperatura máxima promedio diaria es menos de _11 °C_. El mes más frío del año en L

In [ ]:
tools = [search, tool]

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [ ]:
from langchain import hub

In [ ]:
#prompt de la comunidad
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.agents import create_tool_calling_agent

In [ ]:
#Creacion de agentes
agent = create_tool_calling_agent(llm,tools,prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"hola"})



> Entering new AgentExecutor chain...
¡Hola! ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'hola', 'output': '¡Hola! ¿En qué puedo ayudarte hoy?'}

In [ ]:
agent_executor.invoke({"input":"cual es el mejor  superheroe de Marvel y por que?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'List of Marvel superheroes'}`


Page: List of black superheroes
Summary: This is a list of black superheroes that lists characters found in comic books and other media. The characters are superheroes depicted as black people.I couldn't find the information you were looking for. Would you like me to search for the best Marvel superhero based on popularity or any specific criteria?

> Finished chain.


{'input': 'cual es el mejor  superheroe de Marvel y por que?',
 'output': "I couldn't find the information you were looking for. Would you like me to search for the best Marvel superhero based on popularity or any specific criteria?"}

In [ ]:
agent_executor.invoke({"input":"Que temperatura hace en Nueva York?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'temperatura actual en Nueva York'}`


[{'title': 'El clima en Nueva York, Estados Unidos - Tiempo3', 'url': 'https://www.tiempo3.com/north-america/usa/new-york/new-york-city', 'content': 'La temperatura actual en Nueva York es de 26°C. La temperatura máxima en Nueva York para hoy será de 38°C y la mínima se encontrará en torno a los 27°C.', 'score': 0.93425745}, {'title': 'Tiempo actual en Nueva York, NY - AccuWeather', 'url': 'https://www.accuweather.com/es/us/new-york/10021/current-weather/349727', 'content': 'Tiempo actual en Nueva York, NY | AccuWeather\n\n===============\n\n[Volver](https://www.accuweather.com/pwa)\n\n[](https://www.accuweather.com/)[Nueva York, Nueva York ====================== 63°F](https://www.accuweather.com/es/us/new-york/10021/weather-forecast/349727)\n\nUsar la ubicación actual\n\nReciente\n\n[Nueva York Nueva York 63°](https://www.accuweather.com/web-api/three-

{'input': 'Que temperatura hace en Nueva York?',
 'output': 'La temperatura actual en Nueva York es de 26°C. La temperatura máxima para hoy será de 38°C y la mínima se encontrará alrededor de los 27°C. Si deseas más detalles sobre el clima en Nueva York, puedes visitar [Tiempo3 - El clima en Nueva York](https://www.tiempo3.com/north-america/usa/new-york/new-york-city).'}

#Ejemplos de Interacciones

In [ ]:
agent_executor.invoke({"input":"Busca quien fue Alan Turing"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Alan Turing'}`


Page: Alan Turing
Summary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer scAlan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. He was highly influential in the development of theoretical computer science and is widely considered to be the father of theoretical computer science. Turing provided a formalization of the concepts of algorithm and computation with the Turing machine, which can be s

{'input': 'Busca quien fue Alan Turing',
 'output': 'Alan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist. He was highly influential in the development of theoretical computer science and is widely considered to be the father of theoretical computer science. Turing provided a formalization of the concepts of algorithm and computation with the Turing machine, which can be seen as a model of a general-purpose computer.'}

In [ ]:
agent_executor.invoke({"input":"Dime un chiste."})



> Entering new AgentExecutor chain...
¡Claro! Aquí tienes uno:

¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter. ¡Espero que te haya sacado una sonrisa! Si quieres escuchar más chistes, ¡solo dímelo!

> Finished chain.


{'input': 'Dime un chiste.',
 'output': '¡Claro! Aquí tienes uno:\n\n¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter. ¡Espero que te haya sacado una sonrisa! Si quieres escuchar más chistes, ¡solo dímelo!'}

In [ ]:
agent_executor.invoke({"input":"¿Qué está pasando en el mundo de la inteligencia artificial?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'últimas noticias sobre inteligencia artificial'}`


[{'title': 'Últimas noticias de Inteligencia artificial en LA NACION', 'url': 'https://www.lanacion.com.ar/tema/inteligencia-artificial-tid58563/', 'content': '**[Por Victoria Menghini](https://www.lanacion.com.ar/economia/IA/la-justicia-de-eeuu-da-luz-verde-una-demanda-por-danos-severos-provocados-por-inteligencia-nid27052025/ "Por Victoria Menghini")**\n### [Inteligencia artificial](https://www.lanacion.com.ar/tema/inteligencia-artificial-tid58563/ "Noticias de  Inteligencia artificial")\n\n27 de mayo de 2025 [...] **[Por Victoria Mendizábal](https://www.lanacion.com.ar/economia/IA/inteligencia-artificial-y-empleo-oportunidades-desafios-y-la-necesidad-de-adaptarse-nid28052025/ "Por Victoria Mendizábal")**\n### [FuturIA](https://www.lanacion.com.ar/tema/futuria/ "Noticias de  FuturIA")\n\n28 de mayo de 2025 [...] ### [Inteligencia artific

{'input': '¿Qué está pasando en el mundo de la inteligencia artificial?',
 'output': 'Aquí tienes algunas noticias recientes sobre inteligencia artificial:\n\n1. [Últimas noticias de Inteligencia artificial en LA NACION](https://www.lanacion.com.ar/tema/inteligencia-artificial-tid58563/): Se han publicado noticias sobre la inteligencia artificial en LA NACION, incluyendo temas como demandas por daños severos provocados por inteligencia artificial y oportunidades y desafíos en el ámbito laboral.\n\n2. [Todas las noticias sobre inteligencia artificial | Euronews](https://es.euronews.com/tag/inteligencia-artificial): En Euronews, se ha informado sobre cómo la inteligencia artificial está transformando diversas industrias, desde el procesamiento del lenguaje natural hasta la robótica y la investigación médica. También se menciona el aumento de emisiones de grandes empresas tecnológicas debido a la IA.\n\nPuedes acceder a los enlaces para obtener más detalles sobre las noticias mencionadas.

In [ ]:
agent_executor.invoke({"input":"Que esta pasando entre EEUU e Iran?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'EEUU e Iran conflict'}`


[{'title': 'Confrontation With Iran | Global Conflict Tracker', 'url': 'https://www.cfr.org/global-conflict-tracker/conflict/confrontation-between-united-states-and-iran', 'content': 'Background\n\nMore than forty years after the[1979 Iranian Revolution](https://www.brookings.edu/blog/order-from-chaos/2019/01/24/the-iranian-revolution-a-timeline-of-events/ "https://www.brookings.edu/blog/order-from-chaos/2019/01/24/the-iranian-revolution-a-timeline-of-events/"), relations between the United States and Iran remain tense. Iran continues to advance its nuclear program and its support for proxy forces throughout the Middle East, despite U.S. efforts to curtail both. [...] The outbreak of war between Israel, a close U.S. ally, and the Iran-backed militant group Hamas in October 2023 renewed fears of direct conflict between the United States and Iran. Iran-backed proxy f

{'input': 'Que esta pasando entre EEUU e Iran?',
 'output': "The current situation between the United States and Iran remains tense. Iran continues to advance its nuclear program and support proxy forces throughout the Middle East, despite U.S. efforts to curtail both. There have been fears of direct conflict between the United States and Iran, especially after Iran-backed proxy forces increased strikes on U.S. and Israeli targets in Iraq and Syria in response to Israel's military incursion into Gaza.\n\nThe U.S. has indicated that its military action is currently over and has shown no interest in bringing down the government in Tehran. Iran's supreme leader had previously vowed to retaliate against the U.S. if it entered a war, emphasizing the potential irreparable damage that could result from any U.S. military intervention.\n\nThe situation is complex and delicate, with the potential for further escalation. It's important to monitor the developments closely to understand how both co

In [ ]:
agent_executor.invoke({"input":"Quien fue Alan Turing y que fue lo mas grande que hizo, busca en diversas fuentes como paginas webs,etc"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Alan Turing'}`


Page: Alan Turing
Summary: Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer sc
Invoking: `tavily_search_results_json` with `{'query': 'Alan Turing'}`


[{'title': 'Alan Turing - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Alan_Turing', 'content': '**Alan Mathison Turing** ([/ˈtjʊərɪŋ/](https://en.wikipedia.org/wiki/Help:IPA/English "Help:IPA/English"); 23 June 1912\xa0– 7 June 1954) was an English [mathematician](https://en.wikipedia.org/wiki/Mathematician "Mathematician

{'input': 'Quien fue Alan Turing y que fue lo mas grande que hizo, busca en diversas fuentes como paginas webs,etc',
 'output': 'Alan Turing fue un matemático, científico de la computación, lógico, criptoanalista, filósofo y biólogo teórico inglés. Fue altamente influyente en el desarrollo de la ciencia de la computación teórica, proporcionando una formalización de los conceptos de algoritmo y computación con la máquina de Turing, que se puede considerar un modelo de computadora de propósito general. Turing es ampliamente considerado como el padre de la ciencia de la computación teórica.\n\nUna de las contribuciones más importantes de Alan Turing fue su trabajo durante la Segunda Guerra Mundial en el descifrado de las máquinas de cifrado alemanas Enigma. Además, es famoso por desarrollar la primera computadora moderna, detallar el procedimiento conocido como la Prueba de Turing que forma la base de la inteligencia artificial, y por sus contribuciones a las matemáticas, la criptoanálisi

In [ ]:
agent_executor.invoke({"input":"Cuentame otro chiste"})



> Entering new AgentExecutor chain...
¡Claro! Aquí tienes uno:

¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter. ¡Espero que te haya sacado una sonrisa! Si quieres escuchar otro chiste o tienes alguna otra pregunta, ¡estaré encantado de ayudarte!

> Finished chain.


{'input': 'Cuentame otro chiste',
 'output': '¡Claro! Aquí tienes uno:\n\n¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter. ¡Espero que te haya sacado una sonrisa! Si quieres escuchar otro chiste o tienes alguna otra pregunta, ¡estaré encantado de ayudarte!'}

In [ ]:
agent_executor.invoke({"input":"Pero no tienes otros chiste"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'chistes divertidos'}`
responded: ¡Claro que sí! Déjame buscar uno divertido para ti. Un momento, por favor.

[{'title': 'Los chistes cortos más graciosos para reír sin parar con amigos y ...', 'url': 'https://www.clarin.com/internacional/chistes-cortos-graciosos-reir-parar-amigos-familiares_0_lkgB9ybv4I.html?srsltid=AfmBOorj5OSYSgvEIpU6d8at4Bq_CRmDnBqm4mI1Lr4KNu2s1k8ZvmRh', 'content': 'En este contexto, contar chistes es una forma **efectiva y divertida** de compartir momentos inolvidables con amigos y familia. Ya sea un chiste corto para romper el hielo o usando sarcasmo para los adultos o uno inocente para chicos.\n\n## Los mejores chistes con sarcasmo\n\n## Los mejores chistes cortos para reír al instante\n\n## Los mejores chistes malos cortos\n\n## Los mejores chistes para niños sobre frutas\n\n## Los mejores chistes cortos para niños', 'score': 0.67501575}, {'title': '60 chistes cortos

{'input': 'Pero no tienes otros chiste',
 'output': '¡Aquí tienes un chiste corto para sacarte una sonrisa!\n\n- Un pez le pregunta a otro pez: ¿qué hace tu mamá? Este le contesta: Nada, ¿y la tuya qué hace? Nada también.\n\nEspero que te haya gustado. Si quieres más chistes, ¡házmelo saber!'}

In [ ]:
agent_executor.invoke({"input":"si"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'si', 'output': 'Hello! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input":"respondeme tres preguntas, busca en internet 1. ¿Qué otros modelos de colores existen para las imagenes?;¿Qué otros cambios se pueden realizar a las imágenes con OpenCV?; 3. ¿Cuál seria el uso del binary threshold en el procesmiento de imagenes?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '¿Qué otros modelos de colores existen para las imágenes?'}`


[{'title': 'Modelos de color en el procesamiento de imágenes digitales', 'url': 'https://www.toolify.ai/es/ai-news-es/modelos-de-color-en-el-procesamiento-de-imgenes-digitales-1155368', 'content': 'Los modelos de color RGB, CMY y HSI se utilizan en diversas aplicaciones de procesamiento de imágenes. El modelo RGB se utiliza en monitores de video y cámaras de video para representar imágenes en color. El modelo CMY se utiliza en impresoras de color para imprimir imágenes en color. El modelo HSI se utiliza en aplicaciones de procesamiento de imágenes para realizar operaciones como segmentación de objetos, detección de bordes y mejora de la calidad de las imágenes.\n\n## Conclusiones [...] En resumen, el estudio de los fundamentos del procesamiento de imágenes en color nos permite comprender cómo se representan y se manipulan los col

{'input': 'respondeme tres preguntas, busca en internet 1. ¿Qué otros modelos de colores existen para las imagenes?;¿Qué otros cambios se pueden realizar a las imágenes con OpenCV?; 3. ¿Cuál seria el uso del binary threshold en el procesmiento de imagenes?',
 'output': 'Aquí tienes la información encontrada para cada una de tus preguntas:\n\n1. **¿Qué otros modelos de colores existen para las imágenes?**\n   - Existen varios modelos de color utilizados en el procesamiento de imágenes, como RGB, CMY y HSI. Estos modelos se utilizan en diversas aplicaciones para representar y manipular colores en imágenes. Puedes obtener más información en este [enlace](https://www.toolify.ai/es/ai-news-es/modelos-de-color-en-el-procesamiento-de-imgenes-digitales-1155368).\n\n2. **¿Qué otros cambios se pueden realizar a las imágenes con OpenCV?**\n   - Con OpenCV, se pueden realizar una variedad de cambios en las imágenes, como cambiar el tamaño, recortar partes, aplicar filtros, convertir a escala de gr

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'random element from array javascript'}`
responded: ¡Claro! Déjame encontrar un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]. Solo un momento.

[{'title': 'JavaScript - Select a Random Element from JS Array - GeeksforGeeks', 'url': 'https://www.geeksforgeeks.org/javascript/how-to-select-a-random-element-from-array-in-javascript/', 'content': '![geeksforgeeks](https://media.geeksforgeeks.org/gfg-gg-logo.svg)\n\n# JavaScript - Select a Random Element from JS Array\n\nSelecting a random element from array in [JavaScript](https://www.geeksforgeeks.org/javascript/) is done by accessing an element using random index from the array. We can select the random element using the Math.random() function.\n\n![select-random-elements-from-array](https://media.geeksforgeeks.org/wp-content/uploads/20231130124026/select-random-elements-from-arr

{'input': 'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio ',
 'output': '¡He encontrado un saludo aleatorio para ti! El saludo aleatorio del array proporcionado es: "¡Qué tal!". ¡Espero que te haya gustado! ¿Hay algo más en lo que pueda ayudarte?'}

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Hola, ¿cómo estás?'}`
responded: ¡Claro! Déjame encontrar un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]. Solo un momento.

Page: Germán Garmendia
Summary: Germán Alejandro Garmendia Aranis (Spanish pronunciation: [xeɾˈman aleˈxandɾo ɣaɾˈmendja aˈɾanis]; born April 25, 1990) is a Chilean YouTuber, singer-songwriter, comedian and writer. He became famous for his YouTube channel HolaSoyGerman, which uploaded humorous videos about everyday situations. In 2013, he created his gameplay channel, JuegaGerman which, over time, would surpass his previous channel in subscribers.
In 2016, he became the first YouTuber to receive
Invoking: `wikipedia` with `{'query': '¡Buenos días!'}`
responded: ¡Claro! Déjame encontrar un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]. Solo un momento.

Pag

{'input': 'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio ',
 'output': 'He encontrado información sobre cómo crear tarjetas de saludo aleatorias en un proyecto. Puedes aprender a hacerlo y mostrar saludos aleatorios cuando se haga clic en un botón. ¿Te gustaría más detalles sobre este proyecto?'}

In [ ]:
agent_executor.invoke({"input":'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio '})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Hola, ¿cómo estás?'}`
responded: ¡Claro! Déjame encontrar un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]. Solo un momento.

Page: Germán Garmendia
Summary: Germán Alejandro Garmendia Aranis (Spanish pronunciation: [xeɾˈman aleˈxandɾo ɣaɾˈmendja aˈɾanis]; born April 25, 1990) is a Chilean YouTuber, singer-songwriter, comedian and writer. He became famous for his YouTube channel HolaSoyGerman, which uploaded humorous videos about everyday situations. In 2013, he created his gameplay channel, JuegaGerman which, over time, would surpass his previous channel in subscribers.
In 2016, he became the first YouTuber to receive
Invoking: `wikipedia` with `{'query': '¡Buenos días!'}`
responded: ¡Claro! Déjame encontrar un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]. Solo un momento.

Pag

{'input': 'Del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."], devuelveme un saludo aleatorio ',
 'output': '¡Lo siento! Parece que hubo un error al buscar un saludo aleatorio en el array proporcionado. Permíteme intentarlo de nuevo.\n¡Claro! Aquí tienes un saludo aleatorio del siguiente array: ["Hola, ¿cómo estás?", "¡Buenos días!", "¡Qué tal!", "Saludos, humano."]:\n\n¡Qué tal!'}

In [ ]:
agent_executor.invoke({"input":'Cual fue la ultima preegunta que te hice '})



> Entering new AgentExecutor chain...
La última pregunta que me hiciste fue "¿Cuál fue la última pregunta que te hice?" ¡Estoy aquí para ayudarte con cualquier otra pregunta que tengas! ¿En qué más puedo ayudarte hoy?

> Finished chain.


{'input': 'Cual fue la ultima preegunta que te hice ',
 'output': 'La última pregunta que me hiciste fue "¿Cuál fue la última pregunta que te hice?" ¡Estoy aquí para ayudarte con cualquier otra pregunta que tengas! ¿En qué más puedo ayudarte hoy?'}

In [ ]:
agent_executor.invoke({"input":'puedes recordar  nuestro chat?'})



> Entering new AgentExecutor chain...
¡Claro que sí! ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'puedes recordar  nuestro chat?',
 'output': '¡Claro que sí! ¿En qué puedo ayudarte hoy?'}

In [ ]:
agent_executor.invoke({"input":'que  preguntas te he hecho anteriormente'})



> Entering new AgentExecutor chain...
Para responder a tu pregunta, necesitaría acceder al historial de conversación. ¿Te gustaría que busque esa información por ti?

> Finished chain.


{'input': 'que  preguntas te he hecho anteriormente',
 'output': 'Para responder a tu pregunta, necesitaría acceder al historial de conversación. ¿Te gustaría que busque esa información por ti?'}

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '5 + 6 + 7'}`


[{'title': '5,6,7,8 - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/5,6,7,8', 'content': '"**5,6,7,8**" is a song by British group [Steps](https://en.wikipedia.org/wiki/Steps_(pop_group) "Steps (pop group)") from their debut [studio album](https://en.wikipedia.org/wiki/Studio_album "Studio album"), _[Step One](https://en.wikipedia.org/wiki/Step\\_One "Step One")_ (1998). Written by [Barry Upton](https://en.wikipedia.org/wiki/Barry_Upton "Barry Upton") and [Steve Crosby](https://en.wikipedia.org/wiki/Steve_Crosby_(music) "Steve Crosby (music)") and produced by Karl Twigg, Mark Topham [...] quad bikes and dancing in a bar. "5,6,7,8" was performed on [The Ultimate Tour](https://en.wikipedia.org/wiki/The_Ultimate_Tour_(Steps) "The Ultimate Tour (Steps)") in 2012, [Party on the Dancefloor Tour](https://en.wikipedia.org/wiki/Party_on_the_Dancefloor_Tour "Party on the Dancefloor

{'input': 'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros',
 'output': 'La suma de los números 5, 6 y 7 es igual a 18. Puedes encontrar más información sobre el número 5,6,7,8 en [este enlace de Wikipedia](https://en.wikipedia.org/wiki/5,6,7,8).'}

In [ ]:
agent_executor.invoke({"input":'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '5+6+7'}`


[{'title': 'Simplify (5/6)÷(6/7) - Mathway', 'url': 'https://www.mathway.com/popular-problems/Basic%20Math/97622', 'content': 'To divide by a fraction, multiply by its reciprocal. 56⋅76 5 6 ⋅ 7 6. Step 2. Multiply 56⋅76 5', 'score': 0.6420965}, {'title': 'Numbers 5:6-7 - English Standard Version - Bible Gateway', 'url': 'https://www.biblegateway.com/passage/?search=Numbers%205%3A6-7&version=ESV', 'content': 'Speak to the people of Israel, When a man or woman commits any of the sins that people commit by breaking faith with the LORD, and that person.', 'score': 0.5518883}, {'title': '5, 6, 7, 8 - song and lyrics by Steps - Spotify', 'url': 'https://open.spotify.com/track/3oWZwdnnwsCwMukQbPsEiy', 'content': "℗ 1998 Sony Music Entertainment UK Limited [...] ![](https://i.scdn.co/image/ab6761610000f1789628a29f2f8940dc77117e9c)\n![](https://i.scdn.co/image/ab6761610000f178634461910bff

{'input': 'te daré 3 numeros: 5,6,7. Por favor suma esos tres numeros',
 'output': 'La suma de los números 5, 6 y 7 es igual a 18. Puedes verificar más información sobre los números 5, 6, 7 y 8 en [este enlace de Wikipedia](https://en.wikipedia.org/wiki/5,6,7,8).'}

In [ ]:
agent_executor.invoke({"input":'Tienes acceso al directorio de la computadora donde se esta ejecutando este programa?'})



> Entering new AgentExecutor chain...
Lo siento, pero no tengo acceso al directorio de la computadora donde se está ejecutando este programa. Mi función principal es proporcionar información y asistencia en línea. ¿En qué puedo ayudarte hoy?

> Finished chain.


{'input': 'Tienes acceso al directorio de la computadora donde se esta ejecutando este programa?',
 'output': 'Lo siento, pero no tengo acceso al directorio de la computadora donde se está ejecutando este programa. Mi función principal es proporcionar información y asistencia en línea. ¿En qué puedo ayudarte hoy?'}

In [3]:
!pip install langchain-googledrive


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [4]:
import os
from langchain_googledrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain_googledrive.utilities.google_drive import GoogleDriveAPIWrapper

In [15]:
os.environ["GOOGLE_ACCOUNT_FILE"] = "/content/miproyectodrive-464014-c08c3b8e7b82.json"

api_wrapper = GoogleDriveAPIWrapper()


In [16]:
folder_id ="17PjzYJLogP-6UHMKNfQsrXX7-j4u12Pz"

In [17]:
api_wrapper = GoogleDriveAPIWrapper(
    folder_id=folder_id,
    num_results=5,  # ajusta cuántos resultados quieres
    template="gdrive-query-in-folder",  # busca también en el contenido, no solo nombre
)

In [18]:
search_tool = GoogleDriveSearchTool(api_wrapper=api_wrapper)


In [24]:
resultados = search_tool.run("")

print("Archivos accesibles en la carpeta:")
print(resultados)

Archivos accesibles en la carpeta:
No document found
